In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tools.utils import scale_down
from tools.utils import standardize_data, print_genetic_param, print_settings, set_movements, angle_diff, get_distance
from statistics import mean
from random import choice
train_test_options = {"split": 25}

In [2]:
# def check_sampling(x,sampling,data_index):
#     i = 1
#     sampling_acc = True
#     down_limit = (x[1]-sampling)
#     up_limit = (x[1]+sampling)
#     while i<=sampling:
#         if 0 <=down_limit and up_limit < len(data_index):
#             if (data_index[x[1]-i] == True) or (data_index[x[1]+i] == True):
#                 sampling_acc = False
#                 break
#         i = i + 1      
#     return sampling_acc


# def dist_and_bearing_diff(data):
#     all_distances = []
#     bearing_diff = []
#     data_size=len(data)
#     i = 0
#     while i<data_size:
#         if i + 1 >=data_size:
#             break
#         bearing_1, bearing_2 = data["HEADING"].iloc[i], data["HEADING"].iloc[i+1] 
#         bearing_diff.append([abs(bearing_2 - bearing_1),i])
#         lat_1, lon_1, lat_2, lon_2 = data["LAT"].iloc[i], data["LON"].iloc[i], data["LAT"].iloc[i+1], data["LON"].iloc[i+1]
#         all_distances.append(get_distance(lat_1,lon_1,lat_2,lon_2))
#         i = i +1
#     return bearing_diff, all_distances 


# def fitting_indexes(arr,new_size):
#     i = 0
#     r_arr = []
#     while i <len(arr):
#         if i+1 >=len(arr):
#             break
#         r_arr.append(arr[i+1]-arr[i])
#         i = i + 1
#     if len(r_arr) <= 0:
#         return arr
#     mean_space = mean(r_arr)
#     i = 0 
#     while i<len(arr)<new_size:
#         if i+1 >=len(arr):
#             break
#         diff = arr[i+1]-arr[i]
#         if mean_space <= diff:
#             step = int(diff/2)
#             new_index = arr[i] + step
#             arr.insert(i+1,new_index)
#             i = i + 1
#         i = i + 1
#     return arr

# def scalling_down_windowed(data,n_sample,turn_sensitivity=30):
#     if len(data) <= n_sample :
#         return data
#     size_correction = int(len(data) / n_sample) * n_sample
#     data=data[:size_correction]
#     data_size=len(data)
#     data_index = [False for i in range(data_size)]
#     sampling = int((int(data_size/n_sample) * 0.25))
#     labels = data.columns
    
#     temp_idx = [] 
#     final_data = pd.DataFrame(data,columns=labels)
#     final_data.reset_index(drop=True)
    
#     #find the number of bearing differance above the turn sensitivity
#     bearing_diff , all_distances = dist_and_bearing_diff(final_data)
#     mean_dist = mean(all_distances)
#     for idx, x in enumerate(bearing_diff):
#         sampling_acc = check_sampling(x,sampling,data_index)
#         if (x[0] > turn_sensitivity) and (all_distances[idx] > mean_dist/2) and sampling_acc:
#             data_index[x[1]] = True
#             temp_idx.append(x[1])
#     while n_sample > len(temp_idx) and len(temp_idx) > 1 :
#         fitting_indexes(temp_idx,n_sample)

#     for x in temp_idx:
#         data_index[x]=True
        
#     return final_data[data_index]

In [3]:
# if len(glob.glob('ships')) != 0 :
#     files = glob.glob("ships/*.csv")
#     all_ships = [] 
#     for file in files:
#         ship =  pd.read_csv(file)
#         shipname = ship.loc[0]["SHIPNAME"]
#         ship = ship [["TIMESTAMP","LAT","LON","HEADING"]]
#         ship['TIMESTAMP'] = pd.to_datetime(ship['TIMESTAMP'])  
#         ship.sort_values('TIMESTAMP',inplace=True)
#         ship=ship.reset_index(drop=True)
#         n = 500  #chunk row size
#         ship_dfs = [ship[i:i+n] for i in range(0,ship.shape[0],n)]
#         ship_data_chunked = []
#         for x in ship_dfs:
#             x = scale_down(x,train_test_options["split"])
#             x = np.array(x["HEADING"].values.astype(int))
#             ship_data_chunked.append(x)
#         ship_data_chunked = np.array(ship_data_chunked)
#         ship ={"shipname":shipname,"data":ship_data_chunked}
#     all_ships.append(ship)
# #     print(all_ships)

In [4]:
ship =  pd.read_csv("ships/SIEM PILOT.csv")
shipname = ship.loc[0]["SHIPNAME"]
ship = ship [["TIMESTAMP","LAT","LON","HEADING"]]
ship['TIMESTAMP'] = pd.to_datetime(ship['TIMESTAMP'])  
ship.sort_values('TIMESTAMP',inplace=True)
ship=ship.reset_index(drop=True)
# ship = scale_down(ship[0:500],train_test_options["split"])
n = 500  #chunk row size
ship_dfs = [ship[i:i+n] for i in range(0,ship.shape[0],n)]
ship_data_chunked = []
for idx,x in enumerate(ship_dfs):
    x = scale_down(x,train_test_options["split"])
    x = np.array(x["HEADING"].values.astype(int))
    if len(x) == train_test_options["split"]:
        ship_data_chunked.append(x)
ship_data_chunked = np.array(ship_data_chunked)
ship ={"shipname":shipname,"data":ship_data_chunked}
print(ship["data"].shape)


(36, 25)


In [47]:
def data_generation(movement):
    from tools.data_extraction import DataExtractor
    from tools.trajectory_generator import TrajectoryGenerator
    from tools.utils import standardize_data, print_genetic_param, print_settings, set_movements, angle_diff,scale_down, get_distance
    tr_gen_options = {"samples": 25,
                  "freq": 3,
                  "reset_data": True}
    dt_gen_options = {"n_test": 150}

    train_test_options = {"split": 25}

    df_csv_options = {"ts_class": "Bearing"}
    settings = {"trajectory_generator_options": tr_gen_options,
            "data_generation_options": dt_gen_options,
            "train_test_options":train_test_options,
            "define_csvs_option": df_csv_options}
    tr_gen_options = settings["trajectory_generator_options"]
    dt_gen_options = settings["data_generation_options"]
    df_csvs_options = settings["define_csvs_option"]
    train_test_options = settings["train_test_options"]
    first_movement = [movement]
    second_movement = ['random']
    movements = {'first_movement': first_movement,'second_movement': second_movement}
    set_movements(movements)
    tr_gen = TrajectoryGenerator(**tr_gen_options)
    tr_gen.data_generation(**dt_gen_options)
    dex = DataExtractor()
    train_df, test_df = dex.train_test_dataframes(**train_test_options)
    dex.define_csv(**df_csvs_options)
    x_train, y_train, x_test, y_test = dex.load_datasets()
    return x_train, y_train, x_test, y_test




In [ ]:
movement_list = ["spiral_movement_right","step_up_right",
                 "spiral_movement_right","spiral_movement_left",
                 "expanding_square_right","expanding_square_left",
                 "creeping_line_left","creeping_line_right",
                 "sector_pattern_left","sector_pattern_right"]
from sklearn.externals import joblib
from gendis.genetic import GeneticExtractor
predictions = [] 
for x in movement_list:
    print("Experiment for pattern",x)
    with open("models/"+x+".pkl", 'rb') as pickle_file:
        gen_ext = joblib.load( pickle_file)
        x_train, y_train, x_test, y_test = data_generation(x)
        x_test = ship["data"]
        x_train, x_test = standardize_data(x_train, x_test)
        distances_train = gen_ext.transform(x_train)
        lr = LogisticRegression()
        lr.fit(distances_train, y_train)
        distances_test = gen_ext.transform(x_test)
        predictions.append(lr.predict(distances_test))
        pickle_file.close()


2019-05-24 15:06:02.565 | INFO     | tools.trajectory_generator:data_generation:623 - 
 Starting the generator with attributes: 
Original latitude: 37.295493
Original longitude: 23.824322
Initial bearing: 90
Initial speed: 10
Number of samples: 25
Starting time of measurements: 2015-02-01 12:00:00
With initial frequency of collected data: 3 min
and hard reset of data: True
2019-05-24 15:06:02.569 | INFO     | tools.trajectory_generator:data_generation:626 - Create directory 'generator_data' 
2019-05-24 15:06:02.570 | INFO     | tools.trajectory_generator:data_generation:632 - now creating data for movement: spiral_movement_right


Experiment for pattern spiral_movement_right


2019-05-24 15:06:36.376 | INFO     | tools.trajectory_generator:data_generation:637 - now creating data for movement: random
2019-05-24 15:07:07.629 | SUCCESS  | tools.trajectory_generator:data_generation:670 - Done with generator
2019-05-24 15:07:07.631 | INFO     | tools.data_extraction:read_datasets:22 - Reading the data files
2019-05-24 15:07:08.374 | SUCCESS  | tools.data_extraction:read_datasets:51 - Done reading files
2019-05-24 15:07:09.393 | INFO     | tools.data_extraction:define_csv:76 - Creating x_train.csv--y_train.csv and x_test.csv--y_test.csv 
2019-05-24 15:07:09.432 | SUCCESS  | tools.data_extraction:define_csv:101 - Done with train.csv
2019-05-24 15:07:09.445 | INFO     | tools.data_extraction:define_csv:122 - Done with test.csv
2019-05-24 15:07:09.446 | INFO     | tools.data_extraction:load_datasets:129 - Loading the csv files to the appropriate train and test arrays(nparrays)
2019-05-24 15:07:09.467 | SUCCESS  | tools.data_extraction:load_datasets:134 - Done
/home/k

Experiment for pattern step_up_right


2019-05-24 15:07:27.499 | INFO     | tools.trajectory_generator:data_generation:637 - now creating data for movement: random
2019-05-24 15:07:58.539 | SUCCESS  | tools.trajectory_generator:data_generation:670 - Done with generator
2019-05-24 15:07:58.541 | INFO     | tools.data_extraction:read_datasets:22 - Reading the data files
2019-05-24 15:07:59.409 | SUCCESS  | tools.data_extraction:read_datasets:51 - Done reading files
2019-05-24 15:08:00.460 | INFO     | tools.data_extraction:define_csv:76 - Creating x_train.csv--y_train.csv and x_test.csv--y_test.csv 
2019-05-24 15:08:00.491 | SUCCESS  | tools.data_extraction:define_csv:101 - Done with train.csv
2019-05-24 15:08:00.499 | INFO     | tools.data_extraction:define_csv:122 - Done with test.csv
2019-05-24 15:08:00.504 | INFO     | tools.data_extraction:load_datasets:129 - Loading the csv files to the appropriate train and test arrays(nparrays)
2019-05-24 15:08:00.521 | SUCCESS  | tools.data_extraction:load_datasets:134 - Done
/home/k

Experiment for pattern spiral_movement_right


2019-05-24 15:08:41.260 | INFO     | tools.trajectory_generator:data_generation:637 - now creating data for movement: random


In [ ]:
from datetime import datetime
import seaborn as sns
for idx,x in enumerate(movement_list):
    count = 0
    indexes = []
    for ind,i in enumerate(predictions[idx]):
        if i ==0:
            count = count + 1
            indexes.append(ind)
            fig, ax = plt.subplots()
            fig.set_size_inches(30, 20)
            ax.set_ylabel('Latitute', fontsize = 20.0) # Y label
            ax.set_xlabel('Longitude ', fontsize = 20.0) # X label
            start = datetime.strftime(ship_dfs[ind]["TIMESTAMP"].head(1).iloc[0],'%Y-%m-%d %H:%M:%S')
            end = datetime.strftime(ship_dfs[ind]["TIMESTAMP"].tail(1).iloc[0],'%Y-%m-%d %H:%M:%S')
            title = "Experiment:"+x+"\nCHUNK NO: "+str(ind)+"\n"+start+"----"+end
            ax.set_title(title,fontsize = 20.0)
            data=ship_dfs[ind]
#             data=scale_down(ship_dfs[ind],25)
            sns.lineplot(x='LON', y='LAT', data=data, ax=ax,sort=False,marker="o")
    print(count,"times of pattern detection:",x," at indexes ",indexes)

In [50]:
predictions

[array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0]),
 array([0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0,